In [1]:
import pandas as pd
import numpy as np
import sys
# !{sys.executable} -m pip install sklearn
# !{sys.executable} -m pip install environment_kernels
# !{sys.executable} -m pip install msgpack
# !conda update --yes --prefix {sys.prefix} scikit-learn
# import preprocessing from sklearn
# from sklearn import preprocessing
from sklearn.cluster import DBSCAN
from sklearn.impute import SimpleImputer
from sklearn import preprocessing 

In [2]:
people = pd.read_csv("people_and_target.csv")
people = people.drop(['roles', 'normalized name', 'target norm_name','profession', 'family', 'Unnamed: 0', 'Unnamed: 0.1', 'p index', 'date name', 'source', 'target', "target's role"], axis = 1)
people

/Users/anjaliunnithan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,name,processed date,Archive,target_name
0,a-bu-ni-ta,69.1000,unknown,du-u₂-du-ta
1,a-bu-ni-ta,69.1000,unknown,za-la-a
2,du-u₂-du-ta,69.1000,unknown,za-la-a
3,ur-ku₃-nun-na,85.0000,unknown,ur-nig₂
4,ur-ku₃-nun-na,85.0000,unknown,uš-mu-ta
5,ur-ku₃-nun-na,85.0000,domesticated animal,ur-nig₂
6,ur-ku₃-nun-na,85.0000,domesticated animal,uš-mu-ta
7,ur-ku₃-nun-na,85.0000,domesticated animal,ur-nig₂
8,ur-ku₃-nun-na,85.0000,domesticated animal,uš-mu-ta
9,ur-ku₃-nun-na,85.0000,domesticated animal,ur-nig₂


In [3]:
#For now, I am dropping rows that have missing values, since DBSCAN only works when there are no NANs
# people = people.dropna()

Our people dataframe contains columns for the variables we want to use on our clustering algorithm. Our next step is to convert our categorical data into numerical data. We can do this by creating a binary matrix. 

In [4]:
# people_cat = people.select_dtypes(include=[object])
# people_cat['profession'] = people_cat['profession'].astype(str)
# people_cat['roles'] = people_cat['roles'].astype(str)
# people_cat.head(3)

In [5]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
lst = []
for n in people['name'].unique():
    x = people[people['name'] == n]
    flag = False
    for c in x.columns:
        if x[c].isnull().all():
            flag = True
            break
    if flag:
        continue
    lst.append(pd.DataFrame(imp.fit_transform(x)))

In [6]:
len(lst)

784

In [7]:
people_new = pd.DataFrame()
for i in lst:
    people_new = people_new.append(i)

In [8]:
people_new.columns = people.columns

In [9]:
people_new

,name,processed date,Archive,target_name
0,a-bu-ni-ta,69.1,unknown,du-u₂-du-ta
1,a-bu-ni-ta,69.1,unknown,za-la-a
0,du-u₂-du-ta,69.1,unknown,za-la-a
1,du-u₂-du-ta,81.1223,dead animal,nu-ur₂-{d}suen
2,du-u₂-du-ta,81.1223,dead animal,ur-ku₃-nun-na
3,du-u₂-du-ta,80.033,unknown,hu-la-al
4,du-u₂-du-ta,70.03,unknown,ur-{d}nanše
5,du-u₂-du-ta,81.1222,domesticated animal,in-ta-e₃-a-ta
6,du-u₂-du-ta,81.0118,unknown,na-lu₅
7,du-u₂-du-ta,81.0118,domesticated animal,na-lu₅


In [10]:
# TODO: create a LabelEncoder object and fit it to each feature in people_cat


# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()


# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
people_cat_2 = people_new.apply(le.fit_transform)
people_cat_2

,name,processed date,Archive,target_name
0,44,1559,5,116
1,44,1559,5,652
0,201,1559,5,652
1,201,3503,0,451
2,201,3503,0,506
3,201,3433,5,176
4,201,1673,5,574
5,201,3502,1,224
6,201,3476,5,412
7,201,3476,1,412


In [11]:
# TODO: create a OneHotEncoder object, and fit it to all of people_cat

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(people_cat_2)

# 3. Transform
onehotlabels = enc.transform(people_cat_2).toarray()
onehotlabels.shape

# as you can see, you've the same number of rows 891
# but now you've so many more columns due to how we changed all the categorical data into numerical data

/Users/anjaliunnithan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(53243, 5430)

In [12]:
onehotlabels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

We just converted our categorical data into a sparse matrix. We should add this matrix to our quantitative data. 

In [13]:
dates = [[people_new['processed date'].values[i]] for i in np.arange(len(people_new['processed date']))]
people_cat_quant = np.hstack((onehotlabels, dates))
people_cat_quant

array([[ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 69.1   ],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 69.1   ],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 69.1   ],
       ...,
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 65.092 ],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 66.021 ],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 85.0323]])

Our matrix is now ready to be passed through the DBSCAN algorithm. 

Let's try the algorithm with the name "...-a"

In [14]:
people_new[people_new['name'] == 'du-u₂-du-ta']

,name,processed date,Archive,target_name
0,du-u₂-du-ta,69.1,unknown,za-la-a
1,du-u₂-du-ta,81.1223,dead animal,nu-ur₂-{d}suen
2,du-u₂-du-ta,81.1223,dead animal,ur-ku₃-nun-na
3,du-u₂-du-ta,80.033,unknown,hu-la-al
4,du-u₂-du-ta,70.03,unknown,ur-{d}nanše
5,du-u₂-du-ta,81.1222,domesticated animal,in-ta-e₃-a-ta
6,du-u₂-du-ta,81.0118,unknown,na-lu₅
7,du-u₂-du-ta,81.0118,domesticated animal,na-lu₅
8,du-u₂-du-ta,79.1128,domesticated animal,ur-me-me
9,du-u₂-du-ta,80.02,dead animal,{d}šul-gi-iri-mu


In [15]:
people_new.head()

,name,processed date,Archive,target_name
0,a-bu-ni-ta,69.1,unknown,du-u₂-du-ta
1,a-bu-ni-ta,69.1,unknown,za-la-a
0,du-u₂-du-ta,69.1,unknown,za-la-a
1,du-u₂-du-ta,81.1223,dead animal,nu-ur₂-{d}suen
2,du-u₂-du-ta,81.1223,dead animal,ur-ku₃-nun-na


In [16]:
# people.loc[people['name'] == "šu-{d}šul-gi"]
name_example = people_cat_quant[people_cat_quant[:,201] == 1,:]
name_example

array([[ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 69.1   ],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 81.1223],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 81.1223],
       ...,
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 81.1106],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 81.1106],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 81.1226]])

In [17]:
ni = {'name': people.groupby(['name']).size().keys()}
names_and_indices = pd.DataFrame(data=ni)

In [18]:
clustering = DBSCAN(eps=2.5).fit(name_example)
clustering.labels_

array([-1,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0])

In [19]:
a = people_new.loc[people_new['name'] == "du-u₂-du-ta"]
a.reset_index()[pd.Series(clustering.labels_) == 0]

,index,name,processed date,Archive,target_name
1,1,du-u₂-du-ta,81.1223,dead animal,nu-ur₂-{d}suen
2,2,du-u₂-du-ta,81.1223,dead animal,ur-ku₃-nun-na
3,3,du-u₂-du-ta,80.033,unknown,hu-la-al
5,5,du-u₂-du-ta,81.1222,domesticated animal,in-ta-e₃-a-ta
6,6,du-u₂-du-ta,81.0118,unknown,na-lu₅
7,7,du-u₂-du-ta,81.0118,domesticated animal,na-lu₅
8,8,du-u₂-du-ta,79.1128,domesticated animal,ur-me-me
9,9,du-u₂-du-ta,80.02,dead animal,{d}šul-gi-iri-mu
10,10,du-u₂-du-ta,80.0303,dead animal,{d}šul-gi-iri-mu
11,11,du-u₂-du-ta,80.0304,dead animal,ur-nigar{gar}


In [20]:
a.reset_index()[pd.Series(clustering.labels_) == -1]

,index,name,processed date,Archive,target_name
0,0,du-u₂-du-ta,69.1,unknown,za-la-a
4,4,du-u₂-du-ta,70.03,unknown,ur-{d}nanše


In [21]:
people_cat_quant[(people_cat_quant[:,201] == 1) |(people_cat_quant[:,201] == 1) ,:]

array([[ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 69.1   ],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 81.1223],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 81.1223],
       ...,
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 81.1106],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 81.1106],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    , 81.1226]])

In [22]:
people_new.head(180)
people_cat_quant[0:180]
# people_new.groupby('name')['processed date'].apply(list)
# people_new[people_new['name'] == 'ur-ku₃-nun-na']

array([[ 0. ,  0. ,  0. , ...,  0. ,  0. , 69.1],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. , 69.1],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. , 69.1],
       ...,
       [ 0. ,  0. ,  0. , ...,  0. ,  0. , 84.1],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. , 84.1],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. , 84.1]])

In [27]:
clustering = DBSCAN(eps=2.5).fit(people_cat_quant[0:200])
clustering.labels_

array([-1, -1, -1,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [28]:
people_new.head(180).reset_index()[pd.Series(clustering.labels_) == 0]

/Users/anjaliunnithan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,index,name,processed date,Archive,target_name
3,1,du-u₂-du-ta,81.1223,dead animal,nu-ur₂-{d}suen
4,2,du-u₂-du-ta,81.1223,dead animal,ur-ku₃-nun-na
5,3,du-u₂-du-ta,80.033,unknown,hu-la-al
7,5,du-u₂-du-ta,81.1222,domesticated animal,in-ta-e₃-a-ta
8,6,du-u₂-du-ta,81.0118,unknown,na-lu₅
9,7,du-u₂-du-ta,81.0118,domesticated animal,na-lu₅
10,8,du-u₂-du-ta,79.1128,domesticated animal,ur-me-me
11,9,du-u₂-du-ta,80.02,dead animal,{d}šul-gi-iri-mu
12,10,du-u₂-du-ta,80.0303,dead animal,{d}šul-gi-iri-mu
13,11,du-u₂-du-ta,80.0304,dead animal,ur-nigar{gar}


In [29]:
people_new.head(180).reset_index()[pd.Series(clustering.labels_) == -1]

/Users/anjaliunnithan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,index,name,processed date,Archive,target_name
0,0,a-bu-ni-ta,69.1,unknown,du-u₂-du-ta
1,1,a-bu-ni-ta,69.1,unknown,za-la-a
2,0,du-u₂-du-ta,69.1,unknown,za-la-a
6,4,du-u₂-du-ta,70.03,unknown,ur-{d}nanše


In [26]:
people_new.head(180).reset_index()[pd.Series(clustering.labels_) == 1]

,index,name,processed date,Archive,target_name
25,0,ur-ku₃-nun-na,85,unknown,ur-nig₂
26,1,ur-ku₃-nun-na,85,unknown,uš-mu-ta
27,2,ur-ku₃-nun-na,85,domesticated animal,ur-nig₂
28,3,ur-ku₃-nun-na,85,domesticated animal,uš-mu-ta
29,4,ur-ku₃-nun-na,85,domesticated animal,ur-nig₂
30,5,ur-ku₃-nun-na,85,domesticated animal,uš-mu-ta
31,6,ur-ku₃-nun-na,85,domesticated animal,ur-nig₂
32,7,ur-ku₃-nun-na,85,domesticated animal,uš-mu-ta
33,8,ur-ku₃-nun-na,85,domesticated animal,ur-nig₂
34,9,ur-ku₃-nun-na,85,domesticated animal,uš-mu-ta


In [53]:
people_nodate = people_new.drop('processed date', axis = 1)
people_nodate.head()

,name,Archive,target_name
0,a-bu-ni-ta,unknown,du-u₂-du-ta
1,a-bu-ni-ta,unknown,za-la-a
0,du-u₂-du-ta,unknown,za-la-a
1,du-u₂-du-ta,dead animal,nu-ur₂-{d}suen
2,du-u₂-du-ta,dead animal,ur-ku₃-nun-na


In [36]:
# TODO: create a LabelEncoder object and fit it to each feature in people_cat


# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le2 = preprocessing.LabelEncoder()


# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
people_cat_3 = people_nodate.apply(le2.fit_transform)
people_cat_3

,name,Archive,target_name
0,44,5,116
1,44,5,652
0,201,5,652
1,201,0,451
2,201,0,506
3,201,5,176
4,201,5,574
5,201,1,224
6,201,5,412
7,201,1,412


In [37]:
# TODO: create a OneHotEncoder object, and fit it to all of people_cat

# 1. INSTANTIATE
enc2 = preprocessing.OneHotEncoder()

# 2. FIT
enc2.fit(people_cat_3)

# 3. Transform
onehotlabels2 = enc2.transform(people_cat_3).toarray()
onehotlabels2.shape

/Users/anjaliunnithan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(53243, 1653)

In [47]:
onehotlabels2
name_example2 = onehotlabels[onehotlabels2[:,201] == 1,:]
name_example2

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [51]:
clustering3 = DBSCAN(eps=2).fit(name_example2)
clustering3.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [52]:
a = people_nodate.loc[people_nodate['name'] == "du-u₂-du-ta"]
a.reset_index()[pd.Series(clustering3.labels_) == 0]

,index,name,Archive,target_name
0,0,du-u₂-du-ta,unknown,za-la-a
1,1,du-u₂-du-ta,dead animal,nu-ur₂-{d}suen
2,2,du-u₂-du-ta,dead animal,ur-ku₃-nun-na
3,3,du-u₂-du-ta,unknown,hu-la-al
4,4,du-u₂-du-ta,unknown,ur-{d}nanše
5,5,du-u₂-du-ta,domesticated animal,in-ta-e₃-a-ta
6,6,du-u₂-du-ta,unknown,na-lu₅
7,7,du-u₂-du-ta,domesticated animal,na-lu₅
8,8,du-u₂-du-ta,domesticated animal,ur-me-me
9,9,du-u₂-du-ta,dead animal,{d}šul-gi-iri-mu


In [54]:
people_notarget = people_new.drop('target_name', axis = 1)
people_notarget.head()

,name,processed date,Archive
0,a-bu-ni-ta,69.1,unknown
1,a-bu-ni-ta,69.1,unknown
0,du-u₂-du-ta,69.1,unknown
1,du-u₂-du-ta,81.1223,dead animal
2,du-u₂-du-ta,81.1223,dead animal


In [55]:
# TODO: create a LabelEncoder object and fit it to each feature in people_cat


# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le3 = preprocessing.LabelEncoder()

# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
people_cat_4 = people_notarget.apply(le3.fit_transform)
people_cat_4

,name,processed date,Archive
0,44,1559,5
1,44,1559,5
0,201,1559,5
1,201,3503,0
2,201,3503,0
3,201,3433,5
4,201,1673,5
5,201,3502,1
6,201,3476,5
7,201,3476,1


In [56]:
# TODO: create a OneHotEncoder object, and fit it to all of people_cat

# 1. INSTANTIATE
enc3 = preprocessing.OneHotEncoder()

# 2. FIT
enc3.fit(people_cat_4)

# 3. Transform
onehotlabels3 = enc3.transform(people_cat_4).toarray()
onehotlabels3.shape

onehotlabels3


/Users/anjaliunnithan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [57]:
name_example3 = onehotlabels3[onehotlabels3[:,201] == 1,:]
name_example3

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [64]:
clustering4 = DBSCAN(eps=2.5).fit(name_example3)
clustering4.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [65]:
a = people_notarget.loc[people_notarget['name'] == "du-u₂-du-ta"]
a.reset_index()[pd.Series(clustering4.labels_) == 0]

,index,name,processed date,Archive
0,0,du-u₂-du-ta,69.1,unknown
1,1,du-u₂-du-ta,81.1223,dead animal
2,2,du-u₂-du-ta,81.1223,dead animal
3,3,du-u₂-du-ta,80.033,unknown
4,4,du-u₂-du-ta,70.03,unknown
5,5,du-u₂-du-ta,81.1222,domesticated animal
6,6,du-u₂-du-ta,81.0118,unknown
7,7,du-u₂-du-ta,81.0118,domesticated animal
8,8,du-u₂-du-ta,79.1128,domesticated animal
9,9,du-u₂-du-ta,80.02,dead animal


In [77]:
people_noarchive = people_new.drop('Archive', axis = 1)
people_noarchive.head()

# TODO: create a LabelEncoder object and fit it to each feature in people_cat


# # 1. INSTANTIATE
# # encode labels with value between 0 and n_classes-1.
le4 = preprocessing.LabelEncoder()


# # 2/3. FIT AND TRANSFORM
# # use df.apply() to apply le.fit_transform to all columns
people_cat_5 = people_noarchive.apply(le4.fit_transform)


# # TODO: create a OneHotEncoder object, and fit it to all of people_cat

# # 1. INSTANTIATE
enc4 = preprocessing.OneHotEncoder()

# # 2. FIT
enc4.fit(people_cat_5)

# # 3. Transform
onehotlabels4 = enc4.transform(people_cat_5).toarray()

name_example6 = onehotlabels4[onehotlabels4[:,201] == 1,:]

clustering5 = DBSCAN(eps=2).fit(name_example6)
clustering5.labels_

a = people_noarchive.loc[people_noarchive['name'] == "du-u₂-du-ta"]
a.reset_index()[pd.Series(clustering5.labels_) == 0]

/Users/anjaliunnithan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


,index,name,processed date,target_name
0,0,du-u₂-du-ta,69.1,za-la-a
1,1,du-u₂-du-ta,81.1223,nu-ur₂-{d}suen
2,2,du-u₂-du-ta,81.1223,ur-ku₃-nun-na
3,3,du-u₂-du-ta,80.033,hu-la-al
4,4,du-u₂-du-ta,70.03,ur-{d}nanše
5,5,du-u₂-du-ta,81.1222,in-ta-e₃-a-ta
6,6,du-u₂-du-ta,81.0118,na-lu₅
7,7,du-u₂-du-ta,81.0118,na-lu₅
8,8,du-u₂-du-ta,79.1128,ur-me-me
9,9,du-u₂-du-ta,80.02,{d}šul-gi-iri-mu
